In [ ]:
!!pip install pandas_market_calendars

['Collecting pandas_market_calendars',
 '  Downloading pandas_market_calendars-5.1.0-py3-none-any.whl.metadata (9.6 kB)',
 'Requirement already satisfied: pandas>=1.1 in /usr/local/lib/python3.11/dist-packages (from pandas_market_calendars) (2.2.2)',
 'Requirement already satisfied: tzdata in /usr/local/lib/python3.11/dist-packages (from pandas_market_calendars) (2025.2)',
 'Requirement already satisfied: python-dateutil in /usr/local/lib/python3.11/dist-packages (from pandas_market_calendars) (2.9.0.post0)',
 'Collecting exchange-calendars>=3.3 (from pandas_market_calendars)',
 '  Downloading exchange_calendars-4.10.1-py3-none-any.whl.metadata (37 kB)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.11/dist-packages (from exchange-calendars>=3.3->pandas_market_calendars) (2.0.2)',
 'Collecting pyluach (from exchange-calendars>=3.3->pandas_market_calendars)',
 '  Downloading pyluach-2.2.0-py3-none-any.whl.metadata (4.3 kB)',
 'Requirement already satisfied: toolz in /

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from datetime import timedelta
import pandas_market_calendars as mcal  # For trading calendar
import os

# 1. Load the CSV data

data_path = os.path.join("..", "data", "pred_data.csv")
data = pd.read_csv(data_path) # Update path
data = data[['Date', 'Close']]

# 2. Convert 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# 3. Scale the Close prices
scaler = MinMaxScaler(feature_range=(0, 1))
data['Scaled_Close'] = scaler.fit_transform(data[['Close']])

# 4. Prepare the last 60-day sequence for prediction
sequence_length = 60
input_sequence = data['Scaled_Close'].values[-sequence_length:].reshape(1, sequence_length, 1)

# 5. Load the trained LSTM model
model_path = os.path.join("..", "models", "LSTM_model_best.h5")
model = load_model(model_path)


# 6. Make the prediction
predicted_scaled = model.predict(input_sequence)

# 7. Inverse transform to get actual closing price
predicted_price = scaler.inverse_transform([[predicted_scaled[0, 0]]])[0, 0]

# 8. Get the last date and handle non-trading days
last_date = data['Date'].iloc[-1]

# Define NSE holidays (Update for the latest year)
holidays = {
    '2025-01-26', '2025-02-26', '2025-03-14', '2025-03-31', '2025-04-06', '2025-04-10',
    '2025-04-14', '2025-04-18', '2025-05-01', '2025-06-07', '2025-07-06', '2025-08-15',
    '2025-08-27', '2025-10-02', '2025-10-21', '2025-10-22', '2025-11-05', '2025-12-25'
}
holidays = set(pd.to_datetime(list(holidays)))  # Convert to datetime format

# Find the next valid trading day
predicted_date = last_date + timedelta(days=1)
while predicted_date.weekday() >= 5 or predicted_date in holidays:  # Skip weekends & holidays
    predicted_date += timedelta(days=1)

# 9. Print the predicted price
print(f"Predicted closing price for {predicted_date.strftime('%Y-%m-%d')}: {predicted_price:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Predicted closing price for 2025-06-04: 24566.77
